# Lazım olan kitabxanaların əlavə edilməsi

In [1]:
import pandas as pd

In [35]:
data=pd.read_excel('../Downloads/Data_.xlsx')

In [36]:
data.dropna(axis=1,inplace=True)

In [37]:
data.head()

,Unnamed: 0,Date,seher temprator,axsam temp,Seher,Istifade edilen enerji her aile ucun (kwh) per hour,Gunesli saat sayi,solar intensity,solar panel per hour,Solar panel output,Altitute,Windspeed
0,0,2022-07-01,35,30,Laçın,1122,6,1123,345,1552.50,1032,9
1,1,2022-07-02,33,16,Kəlbəcər,276,8,1400,326,1956.00,999,9
2,2,2022-07-03,36,11,Ağdam,873,9,1513,294,1984.50,1045,6
3,3,2022-07-04,35,23,Laçın,1271,7,1336,303,1590.75,955,10
4,4,2022-07-05,33,11,Ağdam,509,7,1565,292,1533.00,1054,6


# Datanın modelləmə üçün hazırlıq mərhələsi

In [38]:
data = pd.get_dummies(data, columns=['Seher'], drop_first=True)

In [39]:
data.head()

,Unnamed: 0,Date,seher temprator,axsam temp,Istifade edilen enerji her aile ucun (kwh) per hour,Gunesli saat sayi,solar intensity,solar panel per hour,Solar panel output,Altitute,...,Seher_Cəbrayıl,Seher_Füzuli,Seher_Kəlbəcər,Seher_Laçın,Seher_Qubadlı,Seher_Xankəndi,Seher_Xocalı,Seher_Xocavənd,Seher_Zəngilan,Seher_Şuşa
0,0,2022-07-01,35,30,1122,6,1123,345,1552.50,1032,...,0,0,0,1,0,0,0,0,0,0
1,1,2022-07-02,33,16,276,8,1400,326,1956.00,999,...,0,0,1,0,0,0,0,0,0,0
2,2,2022-07-03,36,11,873,9,1513,294,1984.50,1045,...,0,0,0,0,0,0,0,0,0,0
3,3,2022-07-04,35,23,1271,7,1336,303,1590.75,955,...,0,0,0,1,0,0,0,0,0,0
4,4,2022-07-05,33,11,509,7,1565,292,1533.00,1054,...,0,0,0,0,0,0,0,0,0,0


In [40]:
data['year']=pd.DatetimeIndex(data['Date']).year
data['month']=pd.DatetimeIndex(data['Date']).month
data['day']=pd.DatetimeIndex(data['Date']).day

In [41]:
data.drop(['Date','Unnamed: 0'],axis=1,inplace=True)

In [9]:
X=data.drop('Solar panel output',axis=1)
y=data['Solar panel output']

# Model qurma

Daha dəqiq nəticələr əldə etmək üçün ensemble learning texnikasından istifadə etdim.

Modeldə də strukturlu datada ən yaxşı nəticə verənlərin "tree based" olduğunu nəzərə alaraq RandomForest və XGboostdan istifadə etdim

In [10]:
from sklearn.model_selection import cross_validate
from xgboost import XGBRegressor
model=XGBRegressor()

In [12]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=125,n_jobs=-1)

In [13]:
from sklearn.ensemble import RandomForestClassifier, VotingRegressor
eclf1 = VotingRegressor(estimators=[ ('XGB', model), ('rf', regr)])
cross_validate(eclf1, X_train, y_train, cv=5, return_train_score=True,n_jobs=-1)
eclf1 = eclf1.fit(X_train, y_train)
pred_ens=eclf1.predict(X_test)

In [14]:
model = model.fit(X_train, y_train)
pred=model.predict(X_test)

In [22]:
X_test['Predicted']=pred

In [23]:
X_test.to_excel('data_predocted.xlsx')

# Api ilə weblə əlaqə qurmaq

In [17]:
import joblib

In [24]:
joblib.dump(model,'Xgboost')

['Xgboost']

In [29]:
from flask import Flask,request
from flask_restful import Resource, Api
import pickle
import pandas as pd
from flask_cors import CORS


app = Flask(__name__)
#
CORS(app)
# creating an API object
api = Api(app)

#prediction api call
class prediction(Resource):
    def get(self,seher_temprator, axsam_temp,aile,Gunesli_saat,intensity, per_hour,Altitute, Windspeed):
        #budget = request.args.get('budget')
        
        print(budget)
        # Let's load the package
        seher_temprator= [int(seher_temprator)]
        axsam_temp=[int(axsam_temp)]
        aile= [int(aile)]
        Gunesli_saat= [int(Gunesli_saat)]
        intensity= [int(intensity)]
        per_hour= [int(per_hour)]
        Altitute= [int(Altitute)]
        Windspeed= [int(Windspeed)]
        dict1={'seher_temprator':seher_temprator, 'axsam_temp':axsam_temp,'aile':aile,'Gunesli_saat':Gunesli_saat,'intensity':intensity, 'per_hour':per_hour,'Altitute':Altitute, 'Windspeed':Windspeed}
        df = pd.DataFrame(dict1)
        model = pickle.load(open('Xgboost.pkl', 'rb'))
        prediction = model.predict(df)
        prediction = int(prediction[0])
        return str(prediction)
        # Let's load the package
        
        return str(prediction)

#data api
class getData(Resource):
    def get(self):
        df = pd.read_excel('../Downloads/data_predocted.xlsx')
        #out = {'key':str}
        res = df.to_json(orient='records')
        #print( res)
        return res

In [30]:
api.add_resource(getData, '/api')

if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Sep/2022 13:36:34] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [11/Sep/2022 13:36:38] "GET /api HTTP/1.1" 200 -
